In [ ]:
!pip install sentence-transformers faiss-cpu pandas transformers
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import re
import json
import google.generativeai as genai
from groq import Groq
import time
import csv

### Generate a Grock API key by following the below instructions:

- Signup/login https://console.groq.com/login
- Create a new api key
- copy the generated key and paste it in below cell.


In [ ]:
grok_api_key = "" # PASTE YOUR API KEY HERE

In [ ]:
# make sure you upload the dataset to colab before running the below,
# ensure correct path is entered
csv_file_path = '/content/chatbot-combined.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the questions
questions = df['question'].tolist()
question_embeddings = model.encode(questions, convert_to_tensor=False)

# Convert embeddings to float32 (required by FAISS)
question_embeddings = np.array(question_embeddings, dtype='float32')

# Create a FAISS index for fast retrieval
index = faiss.IndexFlatL2(question_embeddings.shape[1])  # L2 distance for similarity search
index.add(question_embeddings)  # Add question embeddings to the index


# Define retrieval function to get the closest answer
def get_top_answer(query, model, faiss_index, df, top_k=3):
    # Generate embedding for the input query
    query_embedding = model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding, dtype='float32')

    # Search in the FAISS index
    distances, indices = faiss_index.search(query_embedding, top_k)

    # Retrieve the best matching documents
    retrieved_docs = []
    for idx in indices[0]:
        retrieved_docs.append(df.iloc[idx]['answer'])

    return retrieved_docs

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def GroqChat(question):
    client = Groq(
        api_key=grok_api_key,

    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": question,
            }
        ],
            model = "llama-3.1-70b-versatile"
    )

    cleaned_json_string = chat_completion.choices[0].message.content

    json_str = re.sub(r'}\s*{', '}, {', cleaned_json_string)
    return json_str

In [ ]:
def generate_answer_from_docs(query, retrieved_docs):
    result = []
    if not retrieved_docs:
        return "Don't have an answer for the query."

    context = "\n".join(retrieved_docs)
    result.append(context)
    # print("Context: ", context)

    prompt = f"Answer the following query, based only on the given context.Do not add anything from your previous learnings. Do not state in answer that a context is provided to you. If the context seems irrelavant just say 'I don't have an appropriate answer' query: {query} context: {context}"
    groq_answer = GroqChat(prompt)
    result.append(groq_answer)
    result.append('')
    return result

Ask your question here:

In [ ]:
while True:
  query = input("Enter your query here: ")
  if query == "exit":
    break
  retrieved_docs = list(set(get_top_answer(query, model, index, df)))
  generated_answer = generate_answer_from_docs(query, retrieved_docs)
  print("Llama Answer: ", generated_answer[1])
  print()

Enter your query here: What is Sitare univ?
Llama Answer:  Sitare University is an institution that provides free high-quality Computer Science education to bright students from underprivileged backgrounds, covering food, accommodation, and studies expenses.

Enter your query here: Who is FOunder?
Llama Answer:  The founder of Sitare University is Dr. Amit Singhal, who was earlier Senior Vice President of Google Search.

Enter your query here: Who is dean at SU?
Llama Answer:  Dr. Anuja Agarwal is the founding Dean at Sitare University.

Enter your query here: What is the fee structure?
Llama Answer:  There is no application fee. Additionally, the university provides high-quality education and placements without any charges for the students, as it is free of cost for students from underprivileged backgrounds.

Enter your query here: exit
